# Otimizando os hiper-parâmetros c/ Random Search

## Importando as libs

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

## 1. Aquisição dos dados

In [9]:
number_mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = number_mnist.load_data()

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## 2. Pré-processamento

In [11]:
X_valid, X_train = X_train[:5000] / 255., X_train[5000:] / 255.
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test / 255.

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_valid.shape)
print(y_valid.shape)

(55000, 28, 28)
(55000,)
(10000, 28, 28)
(10000,)
(5000, 28, 28)
(5000,)


## 3. Construindo e treinando o modelo

In [4]:
def build_model(n_hidden=3, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape= input_shape))
    
    if n_hidden > 1:
        for layer in range(n_hidden):
            model.add(keras.layers.Dense(n_neurons, activation="relu"))
            n_neurons = n_neurons/2
    else:
        for layer in range(n_hidden):
            model.add(keras.layers.Dense(n_neurons, activation="relu"))
        
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]) 
    return model

In [5]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [6]:
model.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 

Epoch 1/30
1719/1719 [==============================] - 12s 6ms/step - loss: 1.6751 - accuracy: 0.5188 - val_loss: 0.4830 - val_accuracy: 0.8784
Epoch 2/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4468 - accuracy: 0.8810 - val_loss: 0.3347 - val_accuracy: 0.9056
Epoch 3/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3371 - accuracy: 0.9047 - val_loss: 0.2824 - val_accuracy: 0.9228
Epoch 4/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2851 - accuracy: 0.9194 - val_loss: 0.2528 - val_accuracy: 0.9280
Epoch 5/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2608 - accuracy: 0.9261 - val_loss: 0.2290 - val_accuracy: 0.9360
Epoch 6/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2352 - accuracy: 0.9336 - val_loss: 0.2150 - val_accuracy: 0.9382
Epoch 7/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2249 - accuracy: 0.9355 - val_loss: 0.1999 - val_accuracy

In [7]:
model.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_2 (Dense)              (None, 75)                11325     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                760       
Total params: 292,735
Trainable params: 292,735
Non-trainable params: 0
_________________________________________________________________


## 4. Otimizando os parâmetros com o RandomizedSearchCV

In [8]:
param_distribs = {
    "n_hidden": [3, 4, 5],
    "n_neurons": np.arange(268, 400).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [11]:
rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10), 
                             keras.callbacks.ModelCheckpoint("best_model_mlp_hiperparameters_funil.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/30
1146/1146 [==============================] - 8s 7ms/step - loss: 2.0738 - accuracy: 0.3578 - val_loss: 1.2164 - val_accuracy: 0.7356
Epoch 2/30
1146/1146 [==============================] - 5s 5ms/step - loss: 1.0202 - accuracy: 0.7728 - val_loss: 0.6231 - val_accuracy: 0.8542
Epoch 3/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.5899 - accuracy: 0.8553 - val_loss: 0.4620 - val_accuracy: 0.8810
Epoch 4/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.4622 - accuracy: 0.8773 - val_loss: 0.3901 - val_accuracy: 0.8982
Epoch 5/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.3931 - accuracy: 0.8924 - val_loss: 0.3499 - val_accuracy: 0.9070
Epoch 6/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.3585 - accuracy: 0.8986 - val_loss: 0.3229 - val_accuracy: 0.9138
Epoch 7/30
1146/1146 [==============================] - 5s 4ms/step - loss:

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.0015849365560588501, n_hidden=3, n_neurons=372; total time= 3.3min
Epoch 1/30
1146/1146 [==============================] - 10s 8ms/step - loss: 2.1011 - accuracy: 0.3405 - val_loss: 1.1764 - val_accuracy: 0.7750
Epoch 2/30
1146/1146 [==============================] - 9s 7ms/step - loss: 0.9887 - accuracy: 0.7980 - val_loss: 0.6148 - val_accuracy: 0.8538
Epoch 3/30
1146/1146 [==============================] - 9s 8ms/step - loss: 0.5806 - accuracy: 0.8554 - val_loss: 0.4608 - val_accuracy: 0.8816
Epoch 4/30
1146/1146 [==============================] - 9s 8ms/step - loss: 0.4584 - accuracy: 0.8761 - val_loss: 0.3922 - val_accuracy: 0.8936
Epoch 5/30
1146/1146 [==============================] - 9s 7ms/step - loss: 0.3993 - accuracy: 0.8916 - val_loss: 0.3524 - val_accuracy: 0.9034
Epoch 6/30
1146/1146 [==============================] - 9s 8ms/step - loss: 0.3643 - accuracy: 0.9004 - val_loss: 0.3245 - val_accuracy: 0.9100
Epoch 7/30
1146/1146 [=====================

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.0015849365560588501, n_hidden=3, n_neurons=372; total time= 4.0min
Epoch 1/30
1146/1146 [==============================] - 9s 7ms/step - loss: 2.1340 - accuracy: 0.2964 - val_loss: 1.2885 - val_accuracy: 0.7660
Epoch 2/30
1146/1146 [==============================] - 7s 6ms/step - loss: 1.0682 - accuracy: 0.7918 - val_loss: 0.6344 - val_accuracy: 0.8538
Epoch 3/30
1146/1146 [==============================] - 8s 7ms/step - loss: 0.6038 - accuracy: 0.8580 - val_loss: 0.4636 - val_accuracy: 0.8842
Epoch 4/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.4542 - accuracy: 0.8823 - val_loss: 0.3889 - val_accuracy: 0.8998
Epoch 5/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.3963 - accuracy: 0.8918 - val_loss: 0.3480 - val_accuracy: 0.9086
Epoch 6/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.3626 - accuracy: 0.8989 - val_loss: 0.3238 - val_accuracy: 0.9114
Epoch 7/30
1146/1146 [======================

...

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.005058630952118095, n_hidden=3, n_neurons=345; total time= 1.9min
Epoch 1/30
1146/1146 [==============================] - 5s 4ms/step - loss: 1.5775 - accuracy: 0.5723 - val_loss: 0.4371 - val_accuracy: 0.8854
Epoch 2/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.4116 - accuracy: 0.8854 - val_loss: 0.3137 - val_accuracy: 0.9126
Epoch 3/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.3190 - accuracy: 0.9099 - val_loss: 0.2716 - val_accuracy: 0.9230
Epoch 4/30
1146/1146 [==============================] - 4s 4ms/step - loss: 0.2685 - accuracy: 0.9242 - val_loss: 0.2547 - val_accuracy: 0.9260
Epoch 5/30
1146/1146 [==============================] - 5s 5ms/step - loss: 0.2458 - accuracy: 0.9297 - val_loss: 0.2245 - val_accuracy: 0.9394
Epoch 6/30
1146/1146 [==============================] - 5s 5ms/step - loss: 0.2251 - accuracy: 0.9348 - val_loss: 0.2057 - val_accuracy: 0.9432
Epoch 7/30
1146/1146 [=======================

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.005058630952118095, n_hidden=3, n_neurons=345; total time= 2.0min
Epoch 1/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7941 - accuracy: 0.7524 - val_loss: 0.1873 - val_accuracy: 0.9462
Epoch 2/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.1733 - accuracy: 0.9470 - val_loss: 0.1248 - val_accuracy: 0.9646
Epoch 3/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.1138 - accuracy: 0.9657 - val_loss: 0.1034 - val_accuracy: 0.9704
Epoch 4/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.0812 - accuracy: 0.9756 - val_loss: 0.0834 - val_accuracy: 0.9742
Epoch 5/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.0651 - accuracy: 0.9800 - val_loss: 0.0830 - val_accuracy: 0.9750
Epoch 6/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.0489 - accuracy: 0.9850 - val_loss: 0.0755 - val_accuracy: 0.9764
Epoch 7/30
1719/1719 [=======================

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001DB7C2A8CA0>,
                   param_distributions={'learning_rate': [0.0007604728741086146,
                                                          0.0031744164211451086,
                                                          0.0007462409296099976,
                                                          0.0005055099547700895,
                                                          0.010134172140989477,
                                                          0.0008802115073432119,
                                                          0.001150945479947435,
                                                          0.0011963305185160597,
                                                          0.008256732375...
                                                          0.006493742256474711,
                                                  

In [18]:
rnd_search_cv.best_estimator_

## 5. Avaliando o melhor modelo

In [14]:
best_model = keras.models.load_model("best_model_mlp_hiperparameters_funil.h5")

In [15]:
best_model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_145 (Dense)            (None, 269)               211165    
_________________________________________________________________
dense_146 (Dense)            (None, 134)               36180     
_________________________________________________________________
dense_147 (Dense)            (None, 67)                9045      
_________________________________________________________________
dense_148 (Dense)            (None, 33)                2244      
_________________________________________________________________
dense_149 (Dense)            (None, 10)                340       
Total params: 258,974
Trainable params: 258,974
Non-trainable params: 0
_______________________________________________

In [16]:
# Meta: Acurácia: 0.9798 e Kappa:  0.9775466807563838
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test,y_pred))
print('Kappa: ', cohen_kappa_score(y_test,y_pred))
print('Matriz de confusão: \n', confusion_matrix(y_test,y_pred))

Acurácia:  0.9753
Kappa:  0.9725439788767825
Matriz de confusão: 
 [[ 973    0    1    0    0    1    1    0    3    1]
 [   0 1119    3    2    0    2    2    1    4    2]
 [   4    0 1020    2    1    0    1    1    3    0]
 [   0    0    3  989    0    3    0    5    4    6]
 [   1    0    8    0  938    0    4    4    1   26]
 [   4    0    1   13    0  856    2    1    5   10]
 [   7    2    1    0    5    5  933    0    5    0]
 [   2    4    7    5    0    0    0  998    2   10]
 [   3    1    7    5    2    4    3    3  942    4]
 [   4    2    1    7    5    0    0    4    1  985]]
